In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1618921845797_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2021-01-01' \
and review_length <= 100 \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710235 entries, 0 to 1710234
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UGC_ID           int64 
 1   REVIEW_HEADLINE  object
 2   REVIEW_COMMENTS  object
 3   LANG             object
 4   REVIEW_LENGTH    int64 
 5   FL               int64 
dtypes: int64(3), object(3)
memory usage: 78.3+ MB

In [8]:
#reviews = reviews.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
#reviews['REVIEW_COMMENTS'] = reviews['REVIEW_COMMENTS'].fillna(' ')
#reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' word ' + reviews['REVIEW_COMMENTS']
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                           ALL_TEXT
0  325766472  ...  good bike feels really nice still getting used...
1  325153912  ...  Motion Sickness on this machine You bob up and...
2  325242843  ...                           its a good bike its good
3  330372758  ...  It is a very nice i crashed into a car but the...
4  340220163  ...  This bike is made of 24k gold as much as a dec...

[5 rows x 7 columns]

In [11]:

#reviews['ALL_TEXT_2'] = reviews.apply(lambda x: str(x['ALL_TEXT']).replace(r'[^\w]' , ' '), axis = 1)
#reviews['ALL_TEXT_2'] = reviews.apply(lambda x: re.sub(r'[^\w]', ' ', x['ALL_TEXT']) )

#Look for only words (will still include Chinese characters and such) but removes emojis, symbols, etc
#[^\w] Does not always get all #s though 10/10 removes / and leaves 10 10
def onlywords(text):
    return re.sub(r'[^\w]|[\d]', ' ', text) 


reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['ALL_TEXT_2'] = reviews['ALL_TEXT'].apply(onlywords)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                         ALL_TEXT_2
0  325766472  ...  good bike feels really nice still getting used...
1  325153912  ...  Motion Sickness on this machine You bob up and...
2  325242843  ...                           its a good bike its good
3  330372758  ...  It is a very nice i crashed into a car but the...
4  340220163  ...  This bike is made of   k gold as much as a dec...

[5 rows x 8 columns]

In [13]:
#Apply detect from langdetect package to determine what language is being used
#reviews['ALL_TEXT_2']=reviews['ALL_TEXT_2'].apply(str)
#reviews['PRED_LANG'] = reviews['ALL_TEXT_2'].apply(detect)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [ ]:
#Use this instead of lambda function bc lambda will fail when it encounters just spaces and no words
#This way doesn't error out
texl70 = reviews['ALL_TEXT_2']
langdet = []                                                    

for i in range(len(reviews)):                                         
    try:                                                          
       lang=detect(texl70[i])                                      
    except:                                                       
       lang='no'                 
    langdet.append(lang) 
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
reviews['PRED_LANG'] = langdet

In [ ]:
reviews.head()

In [16]:
reviews['model_FL'] = 0

reviews.loc[ reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Short reviews like "good good" , "Excellent", "i like this" get predicted as "af"
#label as ok bc probably mislabeled as "af" when actually "en"
#reviews.loc[ (reviews['LANG']=='en') & (reviews['PRED_LANG']=='af') , 'model_FL'] = 0

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0

#reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

128714

In [19]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

779

In [20]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6300

In [21]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2853553

In [22]:
FP.head(100)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         UGC_ID                                REVIEW_HEADLINE  \
4     338478727  geeat                                           
41    338479218  all good                                        
89    338470892  Good buy                                        
135   338465717  great                                           
138   338464339  great                                           
144   338473977  EASY!                                           
157   338449696  Good                                            
178   338440091  Cute tic tac toe decor!                         
222   338472156  YES                                             
282   338465969  great                                           
312   338484927  Yws                                             
360   338486326  Producto ok                                     
531   330022627  Really like CK leggings                         
537   330139494  Buy again                                       
558   3300

In [19]:
detect('good good stuff')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [28]:
FP.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 638 to 8681798
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UGC_ID               6456 non-null   int64 
 1   NAME                 6456 non-null   object
 2   REVIEW_HEADLINE      6456 non-null   object
 3   REVIEW_COMMENTS      6456 non-null   object
 4   CR                   6456 non-null   int64 
 5   ALL_TEXT             6456 non-null   object
 6   NAME_1ST_PART_RAW    6456 non-null   object
 7   NAME_1ST_PART        6456 non-null   object
 8   ALL_TEXT_2           6456 non-null   object
 9   ALL_TEXT_3           6456 non-null   object
 10  ALL_TEXT_4           6456 non-null   object
 11  ALL_TEXT_5           6456 non-null   object
 12  ALL_TEXT_6           6456 non-null   object
 13  ALL_TEXT_7           6456 non-null   object
 14  regex_retailer       6456 non-null   object
 15  retailer_not_client  6456 non-null   bool  
 16  m

In [23]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("LANG", StringType(), True)\
,StructField("REVIEW_LENGTH", StringType(), True)\
,StructField("FL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("ALL_TEXT_2", StringType(), True)\
,StructField("PRED_LANG", StringType(), True)\
,StructField("model_FL", IntegerType(), True)])




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
reviews = spark.createDataFrame(reviews, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
reviews.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","FL_langdetect").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://10.101.214.86:18888/sessions/7/statements/26 with error payload: {"msg":"requirement failed: Session isn't active."}


In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…